In [2]:
#################################################################
## Extract best features for peptide detectability prediction
#################################################################

# Gather information from Biopython, Disprot and SeqComplex (Perl)
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO
from Bio import SeqRecord, Seq
from Bio.Alphabet import generic_protein
import re
import pandas as pd
import numpy as np
from sklearn import preprocessing
peptides = pd.read_csv("data/peptides_neighborhood.tsv", sep = "\t")
n = 10
from tqdm import tqdm
import os.path

In [3]:
indices = [idx for idx, e in enumerate(peptides.neigh.values) if not "X" in e]
peptides = peptides.iloc[indices,:]
peptides = peptides.sort_values("neigh")

In [4]:
# peptides = peptides.iloc[:n,:]

In [5]:
peptides.pep = [re.sub("U", "C", e) for e in peptides.pep]
peptides.neigh = [re.sub("U", "C", e) for e in peptides.neigh]

In [6]:
print(peptides.shape)
peptides.head()

(17107, 3)


,pep,neigh,prot
12973,SHHAPMSPGSSGGGGQPIAR,AAAAAAAASGGAQQRSHHAPMSPGSSGGGGQPIARTPQPSSPMDQM...,O14497
5135,GPNGYGFHIHGEK,AAAGAPIPRICCIEKGPNGYGFHIHGEKGKIGQYIRIVEPGSP,O14745
12432,RIGHEIEIR,AAAGPEIAEIQEMWKRIGHEIEIRGKRIEDAIRAQQFYR,O15020
14099,TIGDAVAHIK,AAAGSNSWVQVKTHRTIGDAVAHIKGQGMQIIATHISDNA,P0AGJ2
9779,MAVAESGMGIVEDK,AAAIAAADARIPIAKMAVAESGMGIVEDKVIKNHFASEYIYNAY,P0A9Q7


In [8]:
sequences = peptides.neigh  # add code here
records = (SeqRecord.SeqRecord(Seq.Seq(data=seq, alphabet=generic_protein), description="", id="seq"+ str(index)) for index,seq in enumerate(sequences) )
vl2_input = "data/vl2_input"
vl2_output = "data/vl2_output"


try:
    os.path.mkdir(vl2_input)
except:
    print("Exists")
try:
    os.path.mkdir(vl2_output)
except:
    print("Exists")

for seq in sequences:
    output_handle = open("{}/{}.txt".format(vl2_input, seq), "w")
    output_handle.write(seq)
    output_handle.close()

with open("data/neighborhood.txt", "w") as output_handle:
    [output_handle.write("{}\n".format(e)) for e in sequences]

    
with open("fasta/neighborhood.fasta", "w") as output_handle:
    SeqIO.write(records, output_handle, "fasta")
    
    
sequences = peptides.pep  # add code here
records = (SeqRecord.SeqRecord(Seq.Seq(data=seq, alphabet=generic_protein), description="", id="seq"+ str(index)) for index,seq in enumerate(sequences))
with open("fasta/peptides.fasta", "w") as output_handle:
    SeqIO.write(records, output_handle, "fasta")

Exists
Exists


In [9]:
!cat fasta/neighborhood.fasta | tr  "U" "C"  > temp
!mv temp fasta/neighborhood.fasta

In [10]:
try:
    os.remove("data/hydrophobic_moment_out.txt")
except OSError:
    pass

### Compute hydrophobic_moment

In [11]:
!python /home/antortjim/MEGA/opt/Thesis/hydrophobic_moment/hydrophobic_moment.py -f data/neighborhood.fasta -o data/hydrophobic_moment_out.txt > /dev/null

In [ ]:
# The DataFrame hydrophobic moment will contain several Mean Hydrophobic Moment for the same "neigh" because it's on different windows of it
# We agreggate by taking the mean in a cell below 

In [15]:
hydrophobic_moment = pd.read_csv("data/hydrophobic_moment_out.txt", sep = ",")[["Sequence","Mean Hydrophobic Moment"]]
colnames = hydrophobic_moment.columns.tolist()
colnames[0]= "neigh"
hydrophobic_moment.columns = colnames
print(hydrophobic_moment.shape)
hydrophobic_moment.head()

(432913, 2)


,neigh,Mean Hydrophobic Moment
0,AAAAAAAASGGAQQRSHHAPMSPGSSGGGGQPIARTPQPSSPMDQM...,0.028740
1,AAAAAAAASGGAQQRSHHAPMSPGSSGGGGQPIARTPQPSSPMDQM...,0.028740
2,AAAAAAAASGGAQQRSHHAPMSPGSSGGGGQPIARTPQPSSPMDQM...,0.051497
3,AAAAAAAASGGAQQRSHHAPMSPGSSGGGGQPIARTPQPSSPMDQM...,0.067226
4,AAAAAAAASGGAQQRSHHAPMSPGSSGGGGQPIARTPQPSSPMDQM...,0.084532


In [16]:
peptides[["pep", "neigh"]].head()

,pep,neigh
12973,SHHAPMSPGSSGGGGQPIAR,AAAAAAAASGGAQQRSHHAPMSPGSSGGGGQPIARTPQPSSPMDQM...
5135,GPNGYGFHIHGEK,AAAGAPIPRICCIEKGPNGYGFHIHGEKGKIGQYIRIVEPGSP
12432,RIGHEIEIR,AAAGPEIAEIQEMWKRIGHEIEIRGKRIEDAIRAQQFYR
14099,TIGDAVAHIK,AAAGSNSWVQVKTHRTIGDAVAHIKGQGMQIIATHISDNA
9779,MAVAESGMGIVEDK,AAAIAAADARIPIAKMAVAESGMGIVEDKVIKNHFASEYIYNAY


In [17]:
f = {"Mean Hydrophobic Moment": ["mean"]}
hydrophobic_moment_agg = hydrophobic_moment.groupby("neigh").agg(f)
hydrophobic_moment_agg = pd.DataFrame({"neigh": hydrophobic_moment_agg.index, "hm": hydrophobic_moment_agg.values[:,0]}).sort_values("neigh")
hydrophobic_moment_agg
hydrophobic_moment_agg = pd.merge(hydrophobic_moment_agg, peptides[["pep", "neigh"]], on='neigh', how='outer')
hydrophobic_moment_agg.index = hydrophobic_moment_agg.pep
print(hydrophobic_moment_agg.shape)
hydrophobic_moment_agg.head()

(17107, 3)


,hm,neigh,pep
pep,,,
SHHAPMSPGSSGGGGQPIAR,0.118041,AAAAAAAASGGAQQRSHHAPMSPGSSGGGGQPIARTPQPSSPMDQM...,SHHAPMSPGSSGGGGQPIAR
GPNGYGFHIHGEK,0.193645,AAAGAPIPRICCIEKGPNGYGFHIHGEKGKIGQYIRIVEPGSP,GPNGYGFHIHGEK
RIGHEIEIR,0.408960,AAAGPEIAEIQEMWKRIGHEIEIRGKRIEDAIRAQQFYR,RIGHEIEIR
TIGDAVAHIK,0.239454,AAAGSNSWVQVKTHRTIGDAVAHIKGQGMQIIATHISDNA,TIGDAVAHIK
MAVAESGMGIVEDK,0.086549,AAAIAAADARIPIAKMAVAESGMGIVEDKVIKNHFASEYIYNAY,MAVAESGMGIVEDK


### Compute VL2 disorder

In [266]:
# WARNING!!
############
# It can take 20 mins
# ! ./vl2_disorder.sh vl2_input vl2_output

In [267]:
# vl2_disorder = pd.read_csv(os.path.join(vl2_output, "vl2_output.txt"), sep = "\t", header=None)
# vl2_disorder.columns = ["vl2_disorder", "neigh"]
# vl2_disorder = pd.merge(vl2_disorder, peptides[["pep", "neigh"]], on='neigh', how='outer')
# vl2_disorder.head()

### Compute sequence complexity

In [268]:
# ! perl /home/antortjim/MEGA/opt/Thesis/caballero-SeqComplex-35c8d94/profileComplexSeq.pl
# Does not work

In [18]:
# This cell runs the same code for every row in peptides
# There's >15k rows, look at the tqdm progress bar below
features = pd.DataFrame({
                         "flexibility": [], "aromaticity": [],
                         "instability": [],
                         "composition_G": [],
                         "composition_V": [], "composition_K": [],
                         "length": [], "mass_len_ratio": [],
                        })


for idx, row in tqdm(peptides.iterrows()):
    pep_neigh = row["neigh"]
    pep = row["pep"]
    analysed_pep_neigh = ProteinAnalysis(pep_neigh)
    analysed_pep = ProteinAnalysis(pep)
    
    mw = analysed_pep.molecular_weight()
    
    # feature 
    length = len(pep)
    
    # feature
    mass_len_ratio = mw / length
    
    # features
    composition_G = "G" in pep
    composition_K = "K" in pep
    composition_V = "V" in pep
    
    # feature
    flexibility = np.mean(analysed_pep_neigh.flexibility())
    
    # feature
    aromaticity = np.mean(analysed_pep.aromaticity())
    
    # feature
    instability = np.mean(analysed_pep.instability_index())
    
    # feature
    aa_composition = analysed_pep.get_amino_acids_percent()
    
    
    # compile
    new = pd.DataFrame({
                             "flexibility": [flexibility], "aromaticity": [aromaticity],
                             "instability": [instability], "composition_G": [composition_G],
                             "composition_V": [composition_V], "composition_K": [composition_K],
                             "length": [length], "mass_len_ratio": [mass_len_ratio],
                           })
    new.index = [pep]
    
    features = features.append(pd.DataFrame(data = new))

    # feature
    # compute sequence complexity using SeqComplex for all sequences

17107it [00:50, 338.94it/s]


In [19]:
print(features.shape)
features.head()

(17107, 8)


,aromaticity,composition_G,composition_K,composition_V,flexibility,instability,length,mass_len_ratio
SHHAPMSPGSSGGGGQPIAR,0.000000,1.0,0.0,0.0,1.016665,97.070000,20.0,94.401475
GPNGYGFHIHGEK,0.153846,1.0,1.0,0.0,0.998537,14.123077,13.0,108.654446
RIGHEIEIR,0.000000,1.0,0.0,0.0,1.008597,100.511111,9.0,124.697567
TIGDAVAHIK,0.000000,1.0,1.0,1.0,0.991763,35.960000,10.0,102.417150
MAVAESGMGIVEDK,0.000000,1.0,1.0,1.0,0.999265,27.421429,14.0,102.617879


In [20]:
result = pd.concat([features,
                    hydrophobic_moment_agg["hm"],
#                     vl2_disorder["vl2_disorder"]
                   ], axis=1)

In [21]:
result.head()

,aromaticity,composition_G,composition_K,composition_V,flexibility,instability,length,mass_len_ratio,hm
pep,,,,,,,,,
SHHAPMSPGSSGGGGQPIAR,0.000000,1.0,0.0,0.0,1.016665,97.070000,20.0,94.401475,0.118041
GPNGYGFHIHGEK,0.153846,1.0,1.0,0.0,0.998537,14.123077,13.0,108.654446,0.193645
RIGHEIEIR,0.000000,1.0,0.0,0.0,1.008597,100.511111,9.0,124.697567,0.408960
TIGDAVAHIK,0.000000,1.0,1.0,1.0,0.991763,35.960000,10.0,102.417150,0.239454
MAVAESGMGIVEDK,0.000000,1.0,1.0,1.0,0.999265,27.421429,14.0,102.617879,0.086549


In [22]:
indices = [idx for idx, e in enumerate(result.columns) if re.search("composition", e) is None]
binary_indices = [idx for idx, e in enumerate(result.columns) if re.search("composition", e) is not None]

print(indices)
x = result.values[:,indices]
x_scaled = preprocessing.scale(x)
result.iloc[:,indices] = x_scaled
result.head()

[0, 4, 5, 6, 7, 8]


,aromaticity,composition_G,composition_K,composition_V,flexibility,instability,length,mass_len_ratio,hm
pep,,,,,,,,,
SHHAPMSPGSSGGGGQPIAR,-0.992459,1.0,0.0,0.0,1.648868,1.828800,1.286819,-2.053132,-1.516532
GPNGYGFHIHGEK,0.841374,1.0,1.0,0.0,-0.248549,-0.573767,-0.074351,-0.396156,-0.513330
RIGHEIEIR,-0.992459,1.0,0.0,0.0,0.804389,1.928472,-0.852162,1.468934,2.343718
TIGDAVAHIK,-0.992459,1.0,1.0,1.0,-0.957520,0.058742,-0.657709,-1.121271,0.094526
MAVAESGMGIVEDK,-0.992459,1.0,1.0,1.0,-0.172332,-0.188579,0.120102,-1.097936,-1.934398


In [23]:
result.shape

(17107, 9)

In [24]:
result.to_csv("data/advanced_features.tsv", sep = "\t")